<a href="https://colab.research.google.com/github/jaynetra/AIForHealthCare_Mimic3/blob/main/LLMAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Medical Note Summarization with Gemini and Prompt Engineering

This project aims to develop a robust system for summarizing medical notes using Google
Gemini models and advanced prompt engineering techniques.
By leveraging synthetic data and an iterative refinement process,
the project will deliver high-quality, concise summaries of medical records,
enhancing accessibility and understanding of patient information.

**Key Features**

* **Prompt Engineering:** Employs iterative prompt refinement to guide the model towards producing accurate and informative summaries.

* **Synthetic Data:** Uses synthetic medical data from Hugging Face to train and evaluate the system, ensuring privacy and data security.

**Workflow:**

1. **Data Acquisition:** Load synthetic medical data from Hugging Face Datasets.

4. **Iterative Refinement:**  Continuously evaluate the generated summaries and refine the prompts based on performance analysis.

**TODO**
* **Streamlit Integration:** Deploy the trained model and prompt store within a Streamlit application, enabling users to interact with the system.


* **Interactive Interface:** Provides a user-friendly Streamlit application for querying the model with medical notes and viewing generated summaries.

In [1]:
# pip installs
!pip install datasets
!pip install huggingface_hub

In [2]:
# import packages for models - gemini

!pip install -U -q "google-genai==1.7.0"

In [ ]:
# set up google api key
from google.colab import userdata
userdata.get('GOOGLE_KEY')


In [4]:
# prompt: get some medical notes from hugging face project

from datasets import load_dataset
from google.colab import userdata

# Load the dataset
dataset = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes")

# Access the medical notes
#medical_notes = dataset['train']['text']



In [5]:
# Print the first  medical notes
noteToProcess = dataset['train'][0]['note']


In [6]:
print(noteToProcess)

Discharge Summary:

Patient: 60-year-old male with moderate ARDS from COVID-19

Hospital Course:

The patient was admitted to the hospital with symptoms of fever, dry cough, and dyspnea. During physical therapy on the acute ward, the patient experienced coughing attacks that induced oxygen desaturation and dyspnea with any change of position or deep breathing. To avoid rapid deterioration and respiratory failure, a step-by-step approach was used for position changes. The breathing exercises were adapted to avoid prolonged coughing and oxygen desaturation, and with close monitoring, the patient managed to perform strength and walking exercises at a low level. Exercise progression was low initially but increased daily until hospital discharge to a rehabilitation clinic on day 10.

Clinical Outcome:

The patient was discharged on day 10 to a rehabilitation clinic making satisfactory progress with all symptoms resolved.

Follow-up:

The patient will receive follow-up care at the rehabilita

In [7]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [8]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [9]:
client = genai.Client(api_key=userdata.get('GOOGLE_KEY'))



In [10]:
# Zero shot prompting
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)


prompt = "Summarize the clinical notes"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      prompt])


print(response.text)

A 60-year-old male with moderate COVID-19-induced ARDS was hospitalized for 10 days.  He presented with fever, cough, and dyspnea.  Physical therapy was carefully managed due to his susceptibility to oxygen desaturation and dyspnea with coughing or position changes.  Gradual progress was made in breathing and mobility exercises, culminating in discharge to a rehabilitation clinic on day 10 with resolved symptoms.  He will continue rehabilitation and monitoring at the clinic.  His hospital course was deemed successful with high-quality care.



In [11]:
# few shot prompt
fs_prompt = """
Parse the notes into a valid JSON
example : json
{
  "patient": {
    "age": 60,
    "sex": "male",
    "diagnosis": "Moderate ARDS from COVID-19"
  }
"""
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      fs_prompt])


In [12]:
print(response.text)

```json
{
  "patient": {
    "age": 60,
    "sex": "male",
    "diagnosis": "Moderate ARDS from COVID-19"
  },
  "hospitalCourse": {
    "admissionSymptoms": ["fever", "dry cough", "dyspnea"],
    "physicalTherapy": {
      "challenges": ["coughing attacks inducing oxygen desaturation and dyspnea with position changes and deep breathing"],
      "approach": "step-by-step approach for position changes, adapted breathing exercises to avoid prolonged coughing and oxygen desaturation",
      "exercises": ["strength exercises", "walking exercises"],
      "exerciseProgression": "low initially, increased daily"
    },
    "lengthOfStay": 10
  },
  "clinicalOutcome": {
    "dischargeLocation": "rehabilitation clinic",
    "dischargeStatus": "satisfactory progress, all symptoms resolved"
  },
  "followUp": {
    "location": "rehabilitation clinic",
    "plan": "regular monitoring of progress and further rehabilitation exercises until full recovery",
    "instructions": "Report any new symptoms

In [13]:
fs_prompt = "Parse the notes into a valid JSON and title it Discharge Summary"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      fs_prompt])
print(response.text)

```json
{
  "Discharge Summary": {
    "Patient": {
      "age": 60,
      "sex": "male",
      "diagnosis": "Moderate ARDS from COVID-19"
    },
    "Hospital Course": {
      "admissionSymptoms": ["fever", "dry cough", "dyspnea"],
      "physicalTherapy": {
        "challenges": "Coughing attacks induced oxygen desaturation and dyspnea with position changes or deep breathing.",
        "approach": "Step-by-step approach for position changes, adapted breathing exercises to avoid prolonged coughing and oxygen desaturation.",
        "progress": "Initially low level strength and walking exercises, increased daily until discharge."
      },
      "lengthOfStay": 10
    },
    "Clinical Outcome": {
      "dischargeLocation": "Rehabilitation clinic",
      "dischargeStatus": "Satisfactory progress, all symptoms resolved"
    },
    "Follow-up": {
      "location": "Rehabilitation clinic",
      "plan": "Regular monitoring, further rehabilitation exercises until full recovery. Report any ne

In [14]:
# let us ask some questions
qnPrompt = 'What is the major diagnosis for the patient?'
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      qnPrompt])
print(response.text)

The major diagnosis is **ARDS (Acute Respiratory Distress Syndrome) secondary to COVID-19**.



In [15]:
# Chain of thought promplt
cotPrompt = 'What is the major diagnosis for the patient? Think step by step'
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      cotPrompt])
Markdown(response.text)

Here's a step-by-step breakdown to determine the major diagnosis:

1. **Identify the presenting symptoms:** The patient presented with fever, dry cough, and dyspnea (shortness of breath).

2. **Consider the underlying condition:**  The summary explicitly states the patient had "moderate ARDS from COVID-19".

3. **Understand ARDS:** Acute Respiratory Distress Syndrome (ARDS) is a serious lung condition characterized by fluid buildup in the lungs, making breathing extremely difficult.

4. **The cause:**  The summary clarifies that the ARDS was caused by COVID-19.

5. **Prioritize the diagnosis:** While the patient had fever and cough (symptoms of COVID-19), the major diagnosis driving the hospital course and treatment is the **moderate ARDS secondary to COVID-19**.  The ARDS is the critical condition requiring the intensive care and rehabilitation.

Therefore, the major diagnosis is **moderate ARDS secondary to COVID-19**.


In [16]:
#Thinking mode
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      cotPrompt])


buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())


The major diagnosis for the patient is **moderate ARDS from COVID-19**.

**Step-by-step reasoning:**

1. **Identify the Patient's Condition:** The first line of the discharge summary directly states: "Patient: 60-year-old male with moderate ARDS from COVID-19".
2. **Break Down the Diagnosis:**
    * **ARDS:** This stands for Acute Respiratory Distress Syndrome, a serious lung condition.
    * **COVID-19:** This indicates the cause of the ARDS, which is the Coronavirus Disease 2019.
    * **Moderate:** This describes the severity of the ARDS.

**Conclusion:** The discharge summary explicitly states the patient's condition as "moderate ARDS from COVID-19". This is the primary reason for hospitalization and the focus of the treatment described in the hospital course.

In [17]:
# Experiment Tree of thought
# Get gemini to explain what is a tree of thought
prompt = "Explain what is a tree of thought for a beginner learning LLMs"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      prompt])

In [18]:
print(response.text)

Let's first clarify what a "tree of thought" is *not*.  It's not a direct part of the discharge summary you provided.  That summary describes a patient's medical journey.  A tree of thought is a concept related to how *Large Language Models (LLMs)*, like the one I am, reason and arrive at answers.

Imagine you're trying to solve a complex puzzle. You don't just jump to the solution; you explore different possibilities, discarding some and pursuing others.  This process of exploring and branching out is represented visually as a tree, hence the name "tree of thought."

**For a beginner, think of a tree of thought like this:**

1. **The Root:** This is the initial problem or question the LLM needs to answer.  For example, "What is the best treatment plan for a patient with ARDS?"

2. **Branches:**  These represent different potential approaches or lines of reasoning the LLM explores.  Each branch might represent a different aspect of the problem.  For ARDS, branches could include:  "Oxyg

In [19]:
# explore some questions with reasoning
prompt = 'What is the major diagnosis for the patient? Use tree of thought to conclude and tell how you arrived'
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      prompt])
Markdown(response.text)

**Tree of Thought to Determine Major Diagnosis:**

1. **Initial Symptoms:** The discharge summary begins by stating the patient presented with fever, dry cough, and dyspnea (shortness of breath).  These are common symptoms of various respiratory illnesses.

2. **Diagnosis Confirmation:** The summary explicitly states the patient had "moderate ARDS from COVID-19".  ARDS is acute respiratory distress syndrome.

3. **ARDS Cause:** The summary clarifies that the ARDS was caused by COVID-19.

4. **Hospital Course Details:** The description of the hospital course focuses on managing the complications of ARDS – coughing fits leading to oxygen desaturation and dyspnea, requiring careful physiotherapy adjustments.

5. **Clinical Outcome:** The patient's satisfactory progress and resolution of symptoms reinforce that the ARDS was successfully treated.

6. **Discharge Location:** Discharge to a rehabilitation clinic further supports that the primary focus of care was addressing the lingering effects of ARDS.

**Conclusion:**

Following the tree of thought, the major diagnosis is **ARDS secondary to COVID-19**.  While COVID-19 was the underlying cause, the primary focus of treatment and the significant impact on the patient's health was the development of moderate ARDS. The discharge summary emphasizes the management and improvement of the ARDS, making it the major diagnosis described in the report.


In [20]:
# Teach me the difference between chain of thought and tree of thought and use the data in notetoprocess variable to explain
prompt = 'What is the difference between chain of thought and tree of thought and use the data in notetoprocess variable to explain'
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=noteToProcess.encode('utf-8'),
        mime_type='text/plain',
      ),
      prompt])
Markdown(response.text)

The provided discharge summary doesn't contain information suitable for illustrating the difference between Chain of Thought (CoT) and Tree of Thought (ToT) prompting methods in large language models.  CoT and ToT are techniques used to improve the reasoning capabilities of LLMs, not something that would be directly present in a patient's medical record.

To explain the difference:

* **Chain of Thought (CoT):**  CoT prompting encourages the LLM to generate a step-by-step reasoning process to arrive at an answer.  It's like a linear sequence of thought.  For example, if asked "What is 2 + 2 * 3?", a CoT response might be: "First, we multiply 2 by 3, which is 6. Then we add 2 to 6, resulting in 8.  Therefore, the answer is 8."

* **Tree of Thought (ToT):** ToT expands on CoT by exploring multiple reasoning paths simultaneously.  It's like branching out from a central idea to consider various possibilities.  For the same math problem, a ToT approach might explore different order-of-operations possibilities, eventually converging on the correct answer by evaluating the likelihood and consistency of each path.  It considers several intermediate steps and evaluates which path seems most likely.


The patient's discharge summary describes their *clinical course* and *response to treatment*, not a reasoning process that an LLM would employ.  There's no chain or tree of thoughts present in the data; it's a narrative of events.  To use CoT or ToT, one would need a question that requires reasoning, and then an LLM would produce the chain or tree of thought to answer it.
